In [11]:
import io_utils
corpus_dir="/pi/ai/corpus-zh/Chinese-Names-Corpus-master"
rs=[line.strip() for line in io_utils.lines(corpus_dir+'/Chinese_Names_Corpus（120W）.txt')]
dataset=rs[3:-1]

In [12]:
print(dataset[0], dataset[-1])

阿安 佐治


In [18]:
import numpy

total=len(dataset)
random_rows = numpy.random.randint(total, size=10)
print('pickup', random_rows)
array = numpy.array(dataset)
print(array[random_rows])

pickup [ 697176  287769  393901  388135  867837  249464     241 1028504  981418
  302602]
['孙风雷' '黄洪基' '李克睿' '李基凯' '肖卫华' '何秀杰' '阿丽亚' '张光汉' '尹家燕' '黄彦玲']


In [17]:
from bs4 import BeautifulSoup

def parse_product(text, verbose=True):
    soup = BeautifulSoup(text, 'html.parser')
    props=[]
    images=[]
    for c in soup:
        if verbose:
            print(c.name,'~', c)
        if c.string is not None and len(c.string.strip())>0:
            if verbose:
                print('\t@', c.string)
            props.append(c.string)
    if verbose:
        print('** find images')
    imgs=soup.find_all('img')
    for img in imgs:
        if verbose:
            print(img)
            print('\t', img['src'])
        images.append(img['src'])
    return props, images

text="<img src=\"https://p.qpic.cn/qqumall/0/15435437435c009bbf78be8.jpg/0\" referrerpolicy=\"no-referrer\" /><br>\n        <strong>价格:</strong> 128元"
props, images=parse_product(text, verbose=False)
print(props)
print(images)

['价格:', ' 128元']
['https://p.qpic.cn/qqumall/0/15435437435c009bbf78be8.jpg/0']


In [25]:
def map_list(a):
    b = {a[i]: a[i+1] for i in range(0, len(a), 2)}            
    return b

{**map_list(props), 'image':images[0]}

{'image': 'https://p.qpic.cn/qqumall/0/15435437435c009bbf78be8.jpg/0',
 '价格:': ' 128元'}

In [26]:
def get_product_attrs(entry):
    text=entry['summary']
    props, images=parse_product(text, verbose=False)
    return {**map_list(props), 'image':images[0]}

In [8]:
import dicttoxml
from xml.dom.minidom import parseString
obj = {u'mylist': [u'foo', u'bar', u'baz'], 
       u'mydict': {u'foo': u'bar', u'baz': 1}, 
       u'ok': True}
xml_with_ids = dicttoxml.dicttoxml(obj, 
                                   custom_root='entity-engine-xml',
                                   ids=True)
print(parseString(xml_with_ids).toprettyxml())

<?xml version="1.0" ?>
<entity-engine-xml>
	<mylist id="entity-engine-xml_575821" type="list">
		<item id="mylist_779952_1" type="str">foo</item>
		<item id="mylist_779952_2" type="str">bar</item>
		<item id="mylist_779952_3" type="str">baz</item>
	</mylist>
	<mydict id="entity-engine-xml_285283" type="dict">
		<foo id="mydict_831929" type="str">bar</foo>
		<baz id="mydict_138366" type="int">1</baz>
	</mydict>
	<ok id="entity-engine-xml_784310" type="bool">True</ok>
</entity-engine-xml>



In [10]:
rec={u'foo': u'bar', u'baz': 1}
xml_snippet = dicttoxml.dicttoxml(rec, root=False, attr_type=False)
print(xml_snippet)

b'<foo>bar</foo><baz>1</baz>'


In [1]:
import xml.etree.ElementTree as ET
from sagas.ofbiz.entity_prefabs import EntityPrefabs
xml_file='data/product/ProductPriceTestData.xml'
tree = ET.parse(xml_file)
root = tree.getroot()
ep=EntityPrefabs()
record_set, ids=ep.convert_to_record_set(root)
print(ids)

['UHJvZHVjdDpEZW1vUHJvZHVjdA==', 'UHJvZHVjdDpEZW1vUHJvZHVjdC0x', 'UHJvZHVjdDpEZW1vUHJvZHVjdC0y', 'UHJvZHVjdDpEZW1vUHJvZHVjdC0z', 'UHJvZHVjdEFzc29jOkRlbW9Qcm9kdWN04parRGVtb1Byb2R1Y3QtMuKWq1BST0RVQ1RfVkFSSUFOVOKWqzIwMDYtMDUtMTMgMTI6MDA6MDAuMA==', 'UHJvZHVjdEFzc29jOkRlbW9Qcm9kdWN04parRGVtb1Byb2R1Y3QtMuKWq1BST0RVQ1RfVkFSSUFOVOKWqzIwMDYtMDUtMTMgMTI6MDA6MDAuMA==', 'UHJvZHVjdEFzc29jOkRlbW9Qcm9kdWN04parRGVtb1Byb2R1Y3QtM+KWq1BST0RVQ1RfVkFSSUFOVOKWqzIwMDYtMDUtMTMgMTI6MDA6MDAuMA==', 'UHJvZHVjdFByaWNlOkRlbW9Qcm9kdWN0LTHilqtERUZBVUxUX1BSSUNF4parUFVSQ0hBU0XilqtVU0TilqtfTkFf4parMjAwNi0wNS0xMyAxMjowMDowMC4w', 'UHJvZHVjdFByaWNlOkRlbW9Qcm9kdWN0LTLilqtERUZBVUxUX1BSSUNF4parUFVSQ0hBU0XilqtVU0TilqtfTkFf4parMjAwNi0wNS0xMyAxMjowMDowMC4w', 'UHJvZHVjdFByaWNlOkRlbW9Qcm9kdWN0LTPilqtERUZBVUxUX1BSSUNF4parUFVSQ0hBU0XilqtVU0TilqtfTkFf4parMjAwNi0wNS0xMyAxMjowMDowMC4w']


In [8]:
from values_pb2 import TaStringEntries, TaStringEntriesBatch
from sagas.util.string_util import abbrev
rs=[]
for item in record_set:
    rs.append(item[1])
    print(item[1].entityName) # TaStringEntries
    print('\t', abbrev(item[0]))
batch=TaStringEntriesBatch(records=rs)

Product
	 UHJvZHVjdDpEZW1..
Product
	 UHJvZHVjdDpEZW1..
Product
	 UHJvZHVjdDpEZW1..
Product
	 UHJvZHVjdDpEZW1..
ProductAssoc
	 UHJvZHVjdEFzc29..
ProductAssoc
	 UHJvZHVjdEFzc29..
ProductAssoc
	 UHJvZHVjdEFzc29..
ProductPrice
	 UHJvZHVjdFByaWN..
ProductPrice
	 UHJvZHVjdFByaWN..
ProductPrice
	 UHJvZHVjdFByaWN..


In [10]:
print(len(batch.records))

10


In [4]:
import json_utils
rss_doc=json_utils.read_json_file('./data/rss/westore_new.json')
rss_doc['title']

'Westore'

In [5]:
import datetime
from dateutil.parser import parse
def now():
    return datetime.datetime.now().isoformat()
def now_jdbc():
    return datetime.datetime.now().isoformat(' ')
def to_jdbc(dt):
    return parse(dt).isoformat(' ')
now_jdbc()

'2019-06-09 13:59:47.191664'

In [15]:
def extract_id(id):
    prefix='&path='
    return id[id.index(prefix)+len(prefix):]

id="https://umall.vip.qq.com/v2/?_wv=1025&_wwv=4&product=32482&path=/product/33154"
extract_id(id)

'/product/33154'

In [16]:
from sagas.ofbiz.product.product_data import get_product_attrs
from values_pb2 import TaStringEntries, TaStringEntriesBatch
product_set=[]
for entry in rss_doc['entry']:
    # print(entry["title@zh"], entry['published'])
    data={}
    data['productName']=entry["title@zh"]
    data['createdDate']=now_jdbc()
    data['lastModifiedDate']=now_jdbc()
    data['fromDate']=to_jdbc(entry['published'])
    data['isVirtual']='N'
    data['productTypeId']="FINISHED_GOOD"
    data['productId']=extract_id(entry['id'])
    data['productPricePurposeId']="PURCHASE"
    # data['priceDetailText']=entry['summary']
    attrs=get_product_attrs(entry)
    if "价格:" in attrs:
        data['price']=float(attrs['价格:'].strip().replace('元',''))
        data['currencyUomId']='CNY'
    if "image" in attrs:
        data['largeImageUrl']=attrs['image']
    product_set.append(data)

print(json.dumps(product_set, indent=2, ensure_ascii=False))

[
  {
    "productName": "微信黄脸OK卫衣",
    "createdDate": "2019-06-09 14:46:29.216305",
    "lastModifiedDate": "2019-06-09 14:46:29.216319",
    "fromDate": "2019-06-01 15:21:31.169197",
    "isVirtual": "N",
    "productTypeId": "FINISHED_GOOD",
    "productId": "/product/35022",
    "productPricePurposeId": "PURCHASE",
    "price": 218.0,
    "currencyUomId": "CNY",
    "largeImageUrl": "https://p.qpic.cn/qqumall/0/15501366235c65352fb2c0d.jpg/0"
  },
  {
    "productName": "微信黄脸拉链卫衣",
    "createdDate": "2019-06-09 14:46:29.216782",
    "lastModifiedDate": "2019-06-09 14:46:29.216789",
    "fromDate": "2019-06-01 15:21:31.169210",
    "isVirtual": "N",
    "productTypeId": "FINISHED_GOOD",
    "productId": "/product/34547",
    "productPricePurposeId": "PURCHASE",
    "price": 248.0,
    "currencyUomId": "CNY",
    "largeImageUrl": "https://p.qpic.cn/qqumall/0/15432226195bfbb55bc49cd.jpg/0"
  },
  {
    "productName": "微信黄脸长柄雨伞",
    "createdDate": "2019-06-09 14:46:29.217702",
    "l

In [10]:
def filter_entity(entity_name, props, pickups):
    values={}
    for p in pickups:
        if p in props:
            values[p]=str(props[p])
    record=TaStringEntries(entityName=entity_name, values=values)
    return record
filter_entity('Product', product_set[0], ['productName', 'createdDate', 
                               'lastModifiedDate', 'isVirtual',
                              'productTypeId', 'productId',
                              'largeImageUrl'])

values {
  key: "createdDate"
  value: "2019-06-09 14:43:22.450174"
}
values {
  key: "isVirtual"
  value: "N"
}
values {
  key: "largeImageUrl"
  value: "https://p.qpic.cn/qqumall/0/15501366235c65352fb2c0d.jpg/0"
}
values {
  key: "lastModifiedDate"
  value: "2019-06-09 14:43:22.450192"
}
values {
  key: "productId"
  value: "https://umall.vip.qq.com/v2/?_wv=1025&_wwv=4&product=32482&path=/product/35022"
}
values {
  key: "productName"
  value: "\345\276\256\344\277\241\351\273\204\350\204\270OK\345\215\253\350\241\243"
}
values {
  key: "productTypeId"
  value: "FINISHED_GOOD"
}
entityName: "Product"

In [51]:
filter_entity('ProductPrice', product_set[0], ['createdDate', 
                               'lastModifiedDate', 'fromDate',
                              'price', 'productId',
                              'productPricePurposeId'])

values {
  key: "createdDate"
  value: "2019-06-09 13:46:15.272187"
}
values {
  key: "fromDate"
  value: "2019-06-01 15:21:31.169197"
}
values {
  key: "lastModifiedDate"
  value: "2019-06-09 13:46:15.272202"
}
values {
  key: "price"
  value: "218.0"
}
values {
  key: "productId"
  value: "https://umall.vip.qq.com/v2/?_wv=1025&_wwv=4&product=32482&path=/product/35022"
}
values {
  key: "productPricePurposeId"
  value: "PURCHASE"
}
entityName: "ProductPrice"

In [17]:
import sagas.ofbiz.entities as ee
ent=ee.entity('Product')
filter_entity('ProductPrice', product_set[0], ent.field_names)

values {
  key: "createdDate"
  value: "2019-06-09 14:46:29.216305"
}
values {
  key: "isVirtual"
  value: "N"
}
values {
  key: "largeImageUrl"
  value: "https://p.qpic.cn/qqumall/0/15501366235c65352fb2c0d.jpg/0"
}
values {
  key: "lastModifiedDate"
  value: "2019-06-09 14:46:29.216319"
}
values {
  key: "productId"
  value: "/product/35022"
}
values {
  key: "productName"
  value: "\345\276\256\344\277\241\351\273\204\350\204\270OK\345\215\253\350\241\243"
}
values {
  key: "productTypeId"
  value: "FINISHED_GOOD"
}
entityName: "ProductPrice"

In [4]:
sents='Chtěl / chtěla bych nějaký předkrm.'
if '/' in sents:
    sents=sents.replace(' / ', '/')
    words=[w.split('/')[0] for w in sents.split(' ')]
    sents=' '.join(words)
print(sents)
tokens = re.split(r"([.!?]) /", sents)
print(tokens)

Chtěl bych nějaký předkrm.
['Chtěl bych nějaký předkrm.']


In [7]:
import re
text='Máte oheň? / Můžete mi připálit?'

def alternate(sents):
    if '/' in sents:
        tokens = re.split(r"([.!?]) /", sents)
        if len(tokens)>1:
            sents=' '.join(tokens[0:2])
        else:
            sents=sents.replace(' / ', '/')
            words=[w.split('/')[0] for w in sents.split(' ')]
            sents=' '.join(words)
    return sents

print(alternate('Chtěl / chtěla bych nějaký předkrm.'))
print(alternate('Máte oheň? / Můžete mi připálit?'))

Chtěl bych nějaký předkrm.
Máte oheň ?


In [14]:
sents='‫او (مرد) سر وقت آمد؟ او (مرد) نتوانست سر وقت بیاید.‬'
print('؟ ' in sents, ' ؟' in sents)
sents[:sents.index('؟ ')+2]

True False


'\u202bاو (مرد) سر وقت آمد؟ '

In [6]:
from sagas.nlu.utils import *
sents='‫دخترم نمی‌خواست فوتبال بازی کند.‬'
print(sents)
sents=proc_lang_routines(sents, 'fa')
print(sents)
print(fix_sents(sents, 'fa'))
sents

‫دخترم نمی‌خواست فوتبال بازی کند.‬
‫دخترم نمی‌خواست فوتبال بازی کند.‬
‫دخترم نمی‌خواست فوتبال بازی کند.‬


'\u202bدخترم نمی\u200cخواست فوتبال بازی کند.\u202c'

In [11]:
import difflib
import sys
text="""
خدختر
‫دخترم
""".split('\n')
print(text)
r=difflib.context_diff(text[1], text[2])
sys.stdout.writelines(r)

['', 'خدختر', '\u202bدخترم', '']
*** 
--- 
***************
*** 1,5 ****
! خ  د  خ  ت  ر--- 1,6 ----
! ‫  د  خ  ت  ر+ م

In [12]:
r=difflib.context_diff('sys', 'svs')
sys.stdout.writelines(r)

*** 
--- 
***************
*** 1,3 ****
  s! y  s--- 1,3 ----
  s! v  s